In [89]:
import pandas as pd
import pickle
import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [90]:
df = read_dataframe('./data/fhv_tripdata_2021-01.parquet')

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            1109826 non-null  object        
 4   DOlocationID            1109826 non-null  object        
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1109053 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 76.2+ MB


In [92]:
df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']
df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

In [93]:
# Question 2
average = df.duration.mean()
average

16.2472533682457

In [94]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

df.loc[df['PUlocationID'] == 'nan', 'PUlocationID'] = -1
df.loc[df['DOlocationID'] == 'nan', 'DOlocationID'] = -1

In [95]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1,-1,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1,-1,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1,71.0,None,B00037,9.050000


In [96]:
df['PUlocationID'].value_counts(normalize=True) * 100

-1       83.527328
221.0     0.750568
206.0     0.612438
129.0     0.484671
115.0     0.367805
           ...    
111.0     0.000451
27.0      0.000360
34.0      0.000270
2.0       0.000180
110.0     0.000090
Name: PUlocationID, Length: 262, dtype: float64

In [113]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    print(f'df shape is {df.shape}')
    df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)
    average = df.duration.mean()
    print(f'average duration is {average}')
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df.loc[df['PUlocationID'] == 'nan', 'PUlocationID'] = -1
    df.loc[df['DOlocationID'] == 'nan', 'DOlocationID'] = -1

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [114]:
df_train = read_dataframe('./data/fhv_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/fhv_tripdata_2021-02.parquet')

df shape is (1154112, 7)
average duration is 19.167224093791006
df shape is (1037692, 7)
average duration is 20.706986225199763


In [115]:
train_dicts = df_train[categorical].to_dict(orient='records')

dv = DictVectorizer()


In [116]:
X_train = dv.fit_transform(train_dicts)

In [117]:
X_train.shape

(1109826, 525)

In [118]:
# Training
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519426541347

In [119]:
# Validation
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [120]:
y_val = df_val[target].values

In [121]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.014286601510621